## 0. 라이브러리 설치(세션 재시작 필요)

In [1]:
# 1. 라이브러리 설치 (최초 1회만 실행됨)
!pip install opencv-python mediapipe tensorflow numpy

INFO: pip is looking at multiple versions of mediapipe to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 15.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 4.25.8 which is incompatible.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 4.25.8 which is incompatible.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.8 which is incompatible.


## 1. 라이브러리 로드 및 함수 선언(All_pose_classifier_keras.h5 필요)

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import joblib
import tensorflow as tf
import PIL.Image
import io
import base64
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import os

CURRENT_MODEL_TYPE = 'keras'

def calculate_angle(a, b, c):
    reliability = a[3] * b[3] * c[3]
    if a[3] < 0.5 or b[3] < 0.5 or c[3] < 0.5: return None, reliability
    a_xyz, b_xyz, c_xyz = a[:3], b[:3], c[:3]
    try:
        ba = a_xyz - b_xyz
        bc = c_xyz - b_xyz
        cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
        angle = np.degrees(np.arccos(np.clip(cosine_angle, -1.0, 1.0)))
        return angle, reliability
    except: return None, reliability

def calculate_plane_normal(a, b, c):
    reliability = a[3] * b[3] * c[3]
    if a[3] < 0.5 or b[3] < 0.5 or c[3] < 0.5: return None, reliability
    a_xyz, b_xyz, c_xyz = a[:3], b[:3], c[:3]
    try:
        v1 = b_xyz - a_xyz
        v2 = c_xyz - a_xyz
        normal = np.cross(v1, v2)
        norm_len = np.linalg.norm(normal)
        if norm_len == 0: return None, reliability
        return normal / norm_len, reliability
    except: return None, reliability

def feature_engineering(landmarks):
    """
    [Hybrid Feature Engineering]
    1. Raw 좌표: 어깨 중심 + 어깨 너비 정규화 (Scale Invariant)
    2. 기하학적 특징: 각도(0~1 정규화) + 법선 벡터
    3. 신뢰도 정보 포함

    모든 피처의 스케일을 대략 -1~1 또는 0~1로 맞춰서 결합합니다.
    """
    if landmarks is None: return None

    # ==========================================
    # 1. Raw Coordinates Processing (어깨 중심 & 크기 보정)
    # ==========================================
    lm_copy = landmarks.copy()

    # 인덱스
    L_SHOULDER, R_SHOULDER = 11, 12

    # 기준점(Origin): 어깨 중점
    left_sh = lm_copy[L_SHOULDER, :3]
    right_sh = lm_copy[R_SHOULDER, :3]
    center_point = (left_sh + right_sh) / 2.0

    # 크기 기준(Scale): 어깨 너비
    shoulder_width = np.linalg.norm(left_sh - right_sh)
    scale_factor = shoulder_width if shoulder_width > 1e-6 else 1.0

    # (A) 위치 이동 및 스케일링 -> 결과범위: 대략 -1.0 ~ 1.0
    lm_copy[:, :3] -= center_point
    lm_copy[:, :3] /= scale_factor

    # 1차원으로 펴기 (Visibility 포함 132차원)
    raw_features = lm_copy.flatten()

    # ==========================================
    # 2. Geometric Features (각도 & 법선)
    # ==========================================
    # 인덱스 정의
    NOSE = 0; L_EYE, R_EYE = 2, 5; L_EAR, R_EAR = 7, 8
    L_ELBOW, R_ELBOW = 13, 14; L_WRIST, R_WRIST = 15, 16
    L_HIP, R_HIP = 23, 24

    angle_feats = []
    normal_feats = []
    reliability_feats = []

    # (B) 각도 계산
    angles_to_calc = [
        (landmarks[L_SHOULDER], landmarks[L_ELBOW], landmarks[L_WRIST]),
        (landmarks[R_SHOULDER], landmarks[R_ELBOW], landmarks[R_WRIST]),
        (landmarks[L_ELBOW], landmarks[L_SHOULDER], landmarks[L_HIP]),
        (landmarks[R_ELBOW], landmarks[R_SHOULDER], landmarks[R_HIP]),
        (landmarks[L_EAR], landmarks[L_SHOULDER], landmarks[L_HIP]), # 거북목 확인용
        (landmarks[R_EAR], landmarks[R_SHOULDER], landmarks[R_HIP])
    ]

    for p1, p2, p3 in angles_to_calc:
        ang, rel = calculate_angle(p1, p2, p3)
        # 각도 정규화: 0~180 -> 0.0~1.0
        if ang is not None:
            angle_feats.append(ang / 180.0)
        else:
            angle_feats.append(0.0)
        reliability_feats.append(rel)

    # (C) 법선 벡터 계산 (이미 -1~1 범위)
    # Face Normal
    face_norm, face_rel = calculate_plane_normal(landmarks[NOSE], landmarks[L_EYE], landmarks[R_EYE])
    if face_norm is not None: normal_feats.extend(face_norm)
    else: normal_feats.extend([0.0, 0.0, 0.0])
    reliability_feats.append(face_rel)

    # Chest/Shoulder Normal
    chest_norm, chest_rel = calculate_plane_normal(landmarks[L_SHOULDER], landmarks[R_SHOULDER], landmarks[NOSE])
    if chest_norm is not None: normal_feats.extend(chest_norm)
    else: normal_feats.extend([0.0, 0.0, 0.0])
    reliability_feats.append(chest_rel)

    # ==========================================
    # 3. Feature Concatenation (결합)
    # ==========================================
    # 리스트들을 numpy array로 변환
    geo_vector = np.array(angle_feats + normal_feats + reliability_feats)

    # [최종 결합] Raw 좌표 벡터 + 기하학적 특징 벡터
    # 차원 수: 132 (Raw) + 6 (Angles) + 6 (Normals) + 8 (Reliability) = 152차원
    final_features = np.concatenate([raw_features, geo_vector])

    return final_features

MODEL_PATHS = {
    'rf': 'All_pose_classifier_rf.pkl',
    'svm': 'All_pose_classifier_svm.pkl',
    'keras': 'All_pose_classifier_keras.h5'
}

def load_pose_model(model_type, paths):
    path = paths.get(model_type)

    if not os.path.exists(path):
        print(f"오류: '{path}' 파일이 없습니다. 경로를 확인하세요.")
        return None

    if model_type in ['rf', 'svm']:
        # Scikit-learn 계열 (joblib 사용)
        model = joblib.load(path)
        print(f"[{model_type.upper()}] Scikit-learn 모델 로드 완료!")
        return model

    elif model_type == 'keras':
        # Deep Learning 계열 (Keras 사용)
        model = tf.keras.models.load_model(path)
        print(f"[{model_type.upper()}] Keras 모델 로드 완료!")
        return model

    else:
        print("지원하지 않는 모델 타입입니다.")
        return None

# 모델 로드 실행
model = load_pose_model(CURRENT_MODEL_TYPE, MODEL_PATHS)

# ==========================================
#  예측 수행 함수
# ==========================================
def predict_pose(model, features, model_type):
    """
    모델 타입에 따라 알맞은 전처리와 예측 결과를 반환합니다.
    """
    if model is None or features is None:
        return None

    # 1) Scikit-learn 계열 (RF, SVM)
    if model_type in ['rf', 'svm']:
        # 입력: 2차원 배열 필요 (1, N) -> reshape(1, -1) 권장
        input_data = features.reshape(1, -1)
        prediction = model.predict(input_data)[0] # 결과가 배열로 나오므로 첫 번째 요소 선택
        return prediction

    # 2) Keras 계열 (Deep Learning)
    elif model_type == 'keras':
        # 입력: 반드시 (1, features_len) 형태여야 함
        input_data = features.reshape(1, -1)

        # 예측: 확률 분포 반환 (예: [0.1, 0.8, 0.1])
        prediction_probs = model.predict(input_data, verbose=0)

        # 가장 높은 확률의 인덱스 추출
        prediction = np.argmax(prediction_probs)
        return prediction

# ==========================================
# MediaPipe 설정
# ==========================================
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

print(f"\n현재 설정된 모델: {CURRENT_MODEL_TYPE}")

def js_to_image(js_reply):
  """JS 객체를 OpenCV 이미지로 변환"""
  image_bytes = b64decode(js_reply.split(',')[1])
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  img = cv2.imdecode(jpg_as_np, flags=1)
  return img

def bbox_to_bytes(bbox_array):
  """OpenCV 이미지를 JS 전송용 byte로 변환"""
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  bbox_PIL.save(iobuf, format='png')
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))
  return bbox_bytes

def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom() {
      if (div !== null) {
        return stream;
      }
      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "user"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640;
      captureCanvas.height = 480;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }
      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')
  display(js)

def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

# 2. 메인 실행

In [ ]:
def predict_wrapper(model, features, model_type):
    """
    모델 타입에 따라 적절한 예측 함수를 호출하고,
    (상태 문자열, 'Good'일 확률)을 반환합니다.
    """
    if features is None:
        return "No Pose", 0.0

    # 입력 차원 맞추기 (1, N)
    input_data = features.reshape(1, -1)

    prob_good = 0.0

    try:
        # A. 머신러닝 모델 (RF, SVM)
        if model_type in ['rf', 'svm']:
            # predict_proba 반환값: [[prob_bad, prob_good]]
            probs = model.predict_proba(input_data)[0]
            prob_good = probs[1] # 1번 인덱스가 Good이라고 가정

        # B. 딥러닝 모델 (Keras)
        elif model_type == 'keras':
            # 반환값 형태: [[0.78]] (2차원 배열 안에 값 1개)
            pred = model.predict(input_data, verbose=0)

            prob_good = pred[0][0]

    except Exception as e:
        print(f"Prediction Logic Error: {e}")
        return "Error", 0.0

    # 상태 결정 (임계값 0.5)
    if prob_good > 0.5:
        return "Good", prob_good
    else:
        return "Bad", prob_good

# 스트리밍 시작 (Colab 환경 가정)
try:
    video_stream() 
except NameError:
    print("video_stream() 함수가 정의되지 않았습니다. 이전 셀을 실행했는지 확인하세요.")

label_html = 'Starting Posture Analysis...'
bbox = ''
count = 0

while True:
    # Colab JS 프록시를 통해 프레임 받기
    try:
        js_reply = video_frame(label_html, bbox)
        if not js_reply: break
    except: break

    # 1. 이미지 변환 및 전처리
    img = js_to_image(js_reply["img"])
    img = cv2.flip(img, 1) # 거울 모드
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # 2. MediaPipe 분석
    results = pose.process(img_rgb)

    status = "No Pose"
    prediction_prob = 0.0
    color = (200, 200, 200) # 회색 (Default)

    # 3. 랜드마크 감지 시 처리
    if results.pose_landmarks:
        try:
            # (1) 랜드마크 추출 및 Numpy 변환
            landmarks_list = []
            for lm in results.pose_landmarks.landmark:
                landmarks_list.append([lm.x, lm.y, lm.z, lm.visibility])
            landmarks_arr = np.array(landmarks_list)

            # (2) Feature Engineering (기존 함수 사용)
            features = feature_engineering(landmarks_arr)

            if features is not None:
                # (3) [모듈화된 예측 함수 호출]
                # 여기서 모델 타입에 따라 알아서 처리됩니다.
                status, prediction_prob = predict_wrapper(model, features, CURRENT_MODEL_TYPE)

                # 색상 설정
                if status == "Good":
                    color = (0, 255, 0) # 초록
                elif status == "Bad":
                    color = (0, 0, 255) # 빨강
                else:
                    color = (0, 255, 255) # 노랑 (에러 등)

                label_html = f"[{CURRENT_MODEL_TYPE.upper()}] Status: {status} ({prediction_prob*100:.1f}%)"

            # (4) 시각화: 랜드마크 그리기
            mp_drawing.draw_landmarks(
                img,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=color, thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(255,255,255), thickness=2, circle_radius=2)
            )

            # (5) 시각화: 텍스트 오버레이
            cv2.rectangle(img, (0,0), (300, 60), (245, 117, 16), -1)
            cv2.putText(img, f"{status} {prediction_prob*100:.0f}%", (10, 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            cv2.putText(img, f"Model: {CURRENT_MODEL_TYPE}", (10, 15), # 현재 모델 표시
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), 1)

        except Exception as e:
            print(f"Main Loop Error: {e}")
            pass
    else:
        label_html = 'Pose Not Detected'

    # 4. 출력용 이미지 생성 (RGBA)
    img_rgba = cv2.cvtColor(img, cv2.COLOR_BGR2RGBA)
    bbox_bytes = bbox_to_bytes(img_rgba)
    bbox = bbox_bytes

<IPython.core.display.Javascript object>

/usr/local/lib/python3.12/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
/tmp/ipython-input-3496555551.py:10: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')


KeyboardInterrupt: 